# Tutorial 2: Cycled experiment


**Goal**: To run a cycled data assimilation experiment.
[`cycled_exp.py`](https://github.com/lkugler/DART-WRF/blob/master/generate_free.py) contains an example which will be explained here:

#### Configure your experiment
We start again by configuring `config/cfg.py`.

Then we write a script (or edit an existing one) in the main directory `DART-WRF/`.
`nano new_experiment.py`

---
Any script needs to import some modules:
```python
import os, sys, shutil
import datetime as dt

from dartwrf import utils
from config.cfg import exp
from config.clusters import cluster
```

---
Now there are two options:
- To start a forecast from an existing forecast, i.e. from WRF restart files
- To start a forecast from defined thermodynamic profiles, i.e. from a `wrf_profile`


#### Run a forecast from initial conditions of a previous forecasts
Let's say you want to run a forecast starting at 9 UTC until 12 UTC.
Initial conditions shall be taken from a previous experiment in `/user/test/data/sim_archive/exp_abc` which was initialized at 6 UTC and there are WRF restart files for 9 UTC.
Then the code would be

```python
prior_path_exp = '/user/test/data/sim_archive/exp_abc'
prior_init_time = dt.datetime(2008,7,30,6)
prior_valid_time = dt.datetime(2008,7,30,9)

cluster.setup()

begin = dt.datetime(2008, 7, 30, 9)
end = dt.datetime(2008, 7, 30, 12)

prepare_WRFrundir(begin)

prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time)

run_ENS(begin=begin,  # start integration from here
        end=end,      # integrate until here
        output_restart_interval=9999,  # do not write WRF restart files
        )
```
Note that we use predefined workflow functions like `run_ENS`.


#### Assimilate observations with a prior given by previous forecasts
To assimilate observations at dt.datetime `time` use this command:

```python
prior_path_exp = '/user/test/data/sim_archive/exp_abc'
prior_init_time = dt.datetime(2008,7,30,6)
prior_valid_time = dt.datetime(2008,7,30,9)
time = dt.datetime(2008,7,30,9)  # time of assimilation

assimilate(time, prior_init_time, prior_valid_time, prior_path_exp)
```


#### Update initial conditions from Data Assimilation
In order to continue after assimilation you need the posterior = prior (1) + increments (2)

1. Set prior with this function:

`id = prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time, depends_on=id)`

where path is `str`, times are `dt.datetime`.

2. To update the model state with assimilation increments, you need to update the WRF restart files by running

`id = update_IC_from_DA(time, depends_on=id)`

After this, the wrfrst files are updated with assimilation increments (filter_restart) and copied to the WRF's run directories so you can continue to run the ENS after assimilation using function `run_ENS()`.